# Дистрибутивный анализ *то есть*

In [2]:
import openpyxl
import re
import pymystem3
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords

## Работа с таблицей по векам

In [10]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя листа

In [11]:
m = Mystem()

In [12]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=1001, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' '))
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = re.sub('\[.+?\]', '', item)
        item = re.sub('[Тт]о есть', 'тоесть', item) # \b убирает случаи что есть
        v2.append(item)
values = [v1[i]+v2[i] for i in range(1000)]
print(values[:10])

[' Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, тоесть блюда.   ', ' Само название места происходит от того, что здесь врачевали «косимых» ― тоесть увечных.   ', ' Я даже не могу найти концов, тоесть понять, кто же мне, собственно, должен платить.   ', 'Никаких провокаций ― известные имена-брэнды.  Расчёт на то, что, даже если зритель не шибко приобщён к балету и не знает, кто такая Аньес Летестю, он непременно отреагирует на Opera National de Paris.  Или хотя бы на «Лебединое озеро». Или хотя бы на Paris.  Плюс гарантия качества: звёзды подобраны заслуженные, но при этом, что важно, действующие: спешно доставленные на пароходе прямо из Парижа, поднимешь крышку ― пар! тоесть готовые доказывать свой статус каждым спектаклем.   ', '  «Чтобы не быть выдавленными с рынка, ― говорит академик Алексей Макаров, ― необходимы же

In [13]:
tagged_corp = []
for sentence in values:
    ana = m.analyze(sentence)
    tagged_corp.append(ana)

In [14]:
print(len(tagged_corp))

1000


In [15]:
final = []
count = 1
for sent in tagged_corp:
    sentence = []
    for word in sent:
        if 'analysis' in word and word['analysis'] != []:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            lex = word['analysis'][0]['lex']
            token = word['text']
            if pos == 'V' or pos == 'S' or pos == 'A' or pos == 'ADV':
                sentence.append((token, lex, pos))
    if sentence == []:
        print(sent)
        print(count)
    else:
        final.append(sentence)
    count+=1

In [16]:
with open('sentences.txt', 'w') as fw:
    for sentence in final:
        for word in sentence:
            for part in word:
                fw.write(part + ' ')
        fw.write('\n')

In [ ]:
sentence_counts = []
for sentence in final:
    left_sum = 0
    right_sum = 0
    count_l = 0
    count_r = 0
    ind = 0 # индикатор было ли вхождение то есть
    for word in sentence:
        if word[0] == 'тоесть':
            ind = 1
        if not ind:
            count_l += 1
            left_sum += ?? # тут вектор слова
        elif ind:
            count_r += 1
            left_sum += ?? # тут вектор слова
        
    count = (left_sum, right_sum)
    sentence_counts.append(count)

In [17]:
import re
part_sent = []

# тут я немного криво сохранила текстовый файл с сервера, просто считываю данные номер предложения + косинус
# при этом я выкинула примеры, где вектор хотябы одного контекста nan
with open('part_sim.txt') as fh: 
    lines = fh.readlines()
for line in lines:
    line = line.strip()
    a = re.search(r'([0-9]+)(-?0\.[0-9]+)', line)
    number = a.group(1)
    cos = a.group(2)
    part_sent.append([number, cos])


In [18]:
wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21']
for i in part_sent: # для каждой пары номер-косинус вытаскиваем из таблицы разметку
    i.append(sheet['Q{}'.format(str(int(i[0])+1))].value)
    i.append(sheet['P{}'.format(str(int(i[0])+1))].value)

In [21]:
from collections import Counter

In [40]:
tags_near = [i[2] for i in part_sent if float(i[1]) > 0.6]
tags_near_common = Counter(tags_near)
print(len(tags_near))
tags_near_common

141


Counter({'определение': 12,
         'следствие': 49,
         'дополнение': 14,
         'спецификация': 9,
         'обобщение': 6,
         'переспрос инт': 5,
         'интерпретация': 22,
         'переформулирование': 17,
         'коррекция': 4,
         'пример': 2,
         'переспрос как': 1})

In [42]:
tags_far = [i[2] for i in part_sent if float(i[1]) < 0.4]
tags_far_common = Counter(tags_far)
print(len(tags_far))
tags_far_common

504


Counter({'переформулирование': 62,
         'интерпретация': 132,
         'обобщение': 23,
         'следствие': 110,
         'переспрос инт': 27,
         'дополнение': 40,
         'определение': 35,
         'спецификация': 30,
         'переспрос': 5,
         'коррекция': 18,
         'обр определение': 3,
         'дискурс': 7,
         'переспрос как': 2,
         'пояснение': 5,
         'модальность': 1,
         'пример': 4})